In [22]:
from customer import Customer
import unittest
from unittest.mock import patch, Mock   
import io

In [87]:
# Attempt on integration tests
class TestQuestionnaireModule(unittest.TestCase):
    VALID_PROFILE = {
        "name": 'Francis',
        "age": 20,
        "gender": 'MALE',
    }
    def test_customer_inputs(self):
        profile = {
            "name": "Alice",
            "age": 30,
            "gender" : "Female" # must be MALE, FEMALE
        }
        with self.assertRaises(AssertionError):
            c = Customer(profile)
            
        profile = {
            "name": "Alice",
            "age": '30', # only int
            "gender" : "MALE" 
        }
        with self.assertRaises(AssertionError):
            c = Customer(profile)
            
        profile = {
            "name": "Alice",
            "age":  30,
            "gender" : "MALE" 
        }
        c = Customer(profile)
    
    @patch('builtins.input', side_effect=['12','12','3'])
    @patch('sys.stdout', new_callable=io.StringIO)
    def test_datetime(self, mock_stdout, mock_input):
        customer = Customer(self.VALID_PROFILE)
        customer.ask_duration()
        std_out = mock_stdout.getvalue()
        
        self.assertIn('When would you like to start?', std_out)
        self.assertIn('December', std_out)
        self.assertIn('4 weeks', std_out)
    
    @patch('builtins.input', side_effect=['month','12','12','3'])
    @patch('sys.stdout', new_callable=io.StringIO)
    def test_invalid_datetime(self, mock_stdout, mock_input):
        customer = Customer(self.VALID_PROFILE)
        customer.ask_duration()
        std_out = mock_stdout.getvalue()
        
        self.assertIn('Invalid input.  Please try again.', std_out)
        
    @patch('builtins.input', side_effect=['invalid input','123123','11','a','3'])
    @patch('sys.stdout', new_callable=io.StringIO)
    def test_multiple_choice_questions(self, mock_stdout, mock_input):
        customer = Customer(self.VALID_PROFILE)
        customer.ask_objective()
        std_out = mock_stdout.getvalue()
        
        actual_occurrences = std_out.count('Invalid input.  Please try again.')
        self.assertEqual(4, actual_occurrences//2)
        self.assertIn('Maintain', std_out)
        raise Exception(std_out)